<a href="https://colab.research.google.com/github/nicole-d-ai/Procesamiento_Del_Habla/blob/main/Procesamiento_del_habla(Martin_fierro).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📖 Procesamiento del Habla con Python

**📚 Trabajo Práctico 1 - Martin Fierro 2**

**Nombre**: Nicole Ferreyra

**Profesora**: Ana Laura Diedrichs

**Materia**: Procesamiento del Habla

**Desempeño**: Martin Fierro 2

*Instituto Santo Domingo*

✅Consigna:

Debe trabajar con este PDF
* https://www.argentina.gob.ar/sites/default/files/hernandez_jose_-_el_gaucho_martin_fierro.pdf
* Debe utilizar la librería PDF Plumber para manipular el pdf (https://pypi.org/project/pdfplumber/)
* Explicar el paso a paso de todo lo realizado en el notebook.
* El objetivo es generar una lista en python donde cada elemento es un diccionario con la siguiente información: nro de página texto

# 🔽 Descarga de PDF desde una URL y guardarlo localmente

In [ ]:
import requests

url = "https://www.argentina.gob.ar/sites/default/files/hernandez_jose_-_el_gaucho_martin_fierro.pdf" #PDF que voy a descargar
r = requests.get(url, timeout=60)
r.raise_for_status()

with open("martin_fierro.pdf", "wb") as f:
    f.write(r.content)

print("PDF descargado OK")


PDF descargado OK


In [ ]:
! pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 73.0 MB/s eta 0:00:00


# 📑 Configuración inicial y chequeo del PDF

En esta parte hago la configuración inicial y chequeo del PDF:
* Importo las librerías necesarias
* Preparo las expresiones regulares para detectar números romanos y pies de página.
* Abro el archivo con pdfplumber para verificar cuántas páginas tiene.

In [ ]:
#Importamos la liberia para ABRIR Y LEER PDFs página por página.
import pdfplumber

#Importamos la libreria para trabajar con expreciones regulares (pie de página, romanos)
import regex as re

#Ruta local del PDF
archivo_pdf = "martin_fierro.pdf"

#Patrón precompilado para títulos en números romanos
romanos = re.compile(r'^[IVXLCDM]+$')

#Patrón para “Página N” (con o sin tilde, ignora mayúsc/minúsc)
pie_pagina = re.compile(r'^\s*P(á|a)gina\s+\d+\s*$', re.IGNORECASE)

with pdfplumber.open(archivo_pdf) as pdf:

  #Cantidad de páginas; la imprimo
    print(f"El PDF tiene {len(pdf.pages)} páginas.")

El PDF tiene 52 páginas.


**limpiar_minimo** recibe una línea de texto y devuelve una versión limpia:

(1) si viene None, la reemplaza por "";

(2) convierte el espacio no separable \xa0 a espacio normal;

(3) quita espacios al inicio y al final con .strip().

In [ ]:
def limpiar_minimo(texto):
    return (texto or "").replace("\xa0", " ").strip()

# ✅ Procesamiento de estrofas en una página del PDF

En este bloque defino la función que se encarga de recorrer una página del PDF, limpiar sus líneas y organizarlas en estrofas para poder analizarlas después

* Proceso página por página.

* Filtramos pies de página

* Detectamos romanos como cabecera

* Contamos versos: Cada vez que junto 6, cierro una estrofa; si la página termina antes, igual guardo ese bloque con cant_versos menor a 6, así no se pierde nada.

La función estrofas_en_pagina(archivo_pdf, nro_pagina) construye y devuelve una lista de diccionarios, donde cada diccionario representa una estrofa con su número de página, la cantidad de versos y el texto correspondiente.

In [ ]:
def estrofas_en_pagina(archivo_pdf, nro_pagina):
    lista = []
    acumulador = []
    versos = 0

    with pdfplumber.open(archivo_pdf) as pdf:
        pagina = pdf.pages[nro_pagina - 1]
        texto = pagina.extract_text() or ""
        lineas = [limpiar_minimo(linea) for linea in texto.splitlines()]

    for linea in lineas:
        if not linea or pie_pagina.fullmatch(linea):
            continue

        if romanos.fullmatch(linea):
            if versos > 0:
                lista.append({"nro_pag": nro_pagina,
                              "cant_versos": versos,
                              "texto": "\n".join(acumulador)})
                acumulador, versos = [], 0
            acumulador.append(linea) #Cabecera romana
            continue

        # Línea normal (verso): acumular y contar
        acumulador.append(linea)
        versos += 1

        # Al llegar al tamaño de estrofa (6), cerrar y resetear
        if versos == 6:
            lista.append({"nro_pag": nro_pagina,
                          "cant_versos": 6,
                          "texto": "\n".join(acumulador)})
           acumulador, versos = [], 0

    # Si quedó un resto (<6 versos), guardarlo igual
    if versos > 0:
        lista.append({"nro_pag": nro_pagina,
                      "cant_versos": versos,
                      "texto": "\n".join(acumulador)})

    return lista


# 📑 Presentación de resultados (página 4)

Llamo a estrofas_en_pagina sobre la página 4 y recibo una lista de diccionarios.
Con enumerate(..., start=1) numero cada estrofa en el orden en que aparece.
Para cada estrofa imprimo: número de página, número de estrofa y cantidad de versos; luego muestro el texto completo y dibujo una línea separadora ("-"*60) para leer mejor.


In [ ]:
pagina_4 = estrofas_en_pagina(archivo_pdf, 4)

for i, e in enumerate(pagina_4, start=1):
    print(f"Página {e['nro_pag']} — Estrofa {i} (versos: {e['cant_versos']})")
    print("texto:")
    print(e["texto"])
    print("-" * 60)



Página 4 — Estrofa 1 (versos: 1)
texto:
El Gaucho Martín Fierro
------------------------------------------------------------
Página 4 — Estrofa 2 (versos: 6)
texto:
I
Aquí me pongo a cantar
al compás de la vigüela,
que el hombre que lo desvela
una pena estrordinaria,
como la ave solitaria
con el cantar se consuela.
------------------------------------------------------------
Página 4 — Estrofa 3 (versos: 6)
texto:
Pido a los santos del cielo
que ayuden mi pensamiento:
les pido en este momento
que voy a cantar mi historia
me refresquen la memoria
y aclaren mi entendimiento.
------------------------------------------------------------
Página 4 — Estrofa 4 (versos: 6)
texto:
Vengan santos milagrosos,
vengan todos en mi ayuda,
que la lengua se me añuda
y se me turba la vista;
pido a mi Dios que me asista
en una ocasión tan ruda.
------------------------------------------------------------
Página 4 — Estrofa 5 (versos: 6)
texto:
Yo he visto muchos cantores,
con famas bien otenidas
y que des

In [ ]:
todas = []
with pdfplumber.open(archivo_pdf) as pdf:
    for n in range(1, len(pdf.pages) + 1):
        todas.extend(estrofas_en_pagina(archivo_pdf, n))

print(f"Total de estrofas: {len(todas)}")
for i, e in enumerate(todas, start=1):
    print(f"{i:03d} [pág {e['nro_pag']}] (versos: {e['cant_versos']})")
    print(e["texto"])
    print("-" * 40)


Total de estrofas: 402
001 [pág 1] (versos: 4)
El gaucho
Martín Fierro
José Hernández
Recursos de dominio público
----------------------------------------
002 [pág 2] (versos: 6)
José Hernández
El Gaucho Martín Fierro
José Hernández (1834 - 1886)
Imágenes de dominio público. Fuente:
http://upload.wikimedia.org/wikipedia/commons/8/8c/Jos%C3%A9_Hern%C3%A1ndez_Argentino.jpg
http://upload.wikimedia.org/wikipedia/commons/3/30/El_Gaucho_Mart%C3%ADn_Fierro_2.jpg
----------------------------------------
003 [pág 3] (versos: 2)
http://upload.wikimedia.org/wikipedia/commons/4/44/El_Gaucho_Mart%C3%ADn_Fierro_3.jpg
http://upload.wikimedia.org/wikipedia/commons/4/47/El_Gaucho_Mart%C3%ADn_Fierro_1.jpg
----------------------------------------
004 [pág 4] (versos: 1)
El Gaucho Martín Fierro
----------------------------------------
005 [pág 4] (versos: 6)
I
Aquí me pongo a cantar
al compás de la vigüela,
que el hombre que lo desvela
una pena estrordinaria,
como la ave solitaria
con el cantar se consuel

# **✅ En conclusión: el trabajo logra transformar un PDF crudo en una estructura ordenada de estrofas listas para analizar o mostrar. Se cumplieron los objetivos de la consigna: obtener una lista de diccionarios con el número de página y el texto de cada estrofa, filtrando correctamente los elementos no deseados.**